In [1]:
import tkinter as tk
from tkinter import messagebox

# -------------------------- Base de conocimiento de vehículos ---------------- #
# Base de conocimientos de Vehiculos
# Reglas para razonamiento hacia delante
reglas_adelante = [
    # Reglas para determinar el tipo de vehículo basado en el número de ruedas
    {'condiciones': lambda hechos: hechos['num_ruedas'] < 4, 
    'accion': lambda hechos: hechos.update({'tipoDeVehiculo': 'cycle'})},
    
    {'condiciones': lambda hechos: hechos['num_ruedas'] == 4 and hechos['motor'] == True,
    'accion': lambda hechos: hechos.update({'tipoDeVehiculo': 'automobil'})},
    
    # Reglas para identificar vehículos de tipo bicicleta
    {'condiciones': lambda hechos: hechos['tipoDeVehiculo'] == 'cycle' and hechos['num_ruedas'] == 2 and hechos['motor'] == False,
    'accion': lambda hechos: hechos.update({'vehiculo': 'Bicicleta'})},
    
    {'condiciones': lambda hechos: hechos['tipoDeVehiculo'] == 'cycle' and hechos['num_ruedas'] == 3 and hechos['motor'] == False,
    'accion': lambda hechos: hechos.update({'vehiculo': 'Triciclo'})},
    
    {'condiciones': lambda hechos: hechos['tipoDeVehiculo'] == 'cycle' and hechos['num_ruedas'] == 2 and hechos['motor'] == True,
    'accion': lambda hechos: hechos.update({'vehiculo': 'Motocicleta'})},
    
    # Reglas para identificar vehículos de tipo automobile
    {'condiciones': lambda hechos: hechos['tipoDeVehiculo'] == 'automobil' and hechos['tamanio'] == 'grande' and hechos['num_puertas'] == 1,
    'accion': lambda hechos: hechos.update({'vehiculo': 'Formula 1'})},

    {'condiciones': lambda hechos: hechos['tipoDeVehiculo'] == 'automobil' and hechos['tamanio'] == 'pequenio' and hechos['num_puertas'] == 2,
    'accion': lambda hechos: hechos.update({'vehiculo': 'Carro deportivo'})},
    
    {'condiciones': lambda hechos: hechos['tipoDeVehiculo'] == 'automobil' and hechos['tamanio'] == 'mediano' and hechos['num_puertas'] == 4,
    'accion': lambda hechos: hechos.update({'vehiculo': 'Sedan'})},
    
    {'condiciones': lambda hechos: hechos['tipoDeVehiculo'] == 'automobil' and hechos['tamanio'] == 'mediano' and hechos['num_puertas'] == 3,
    'accion': lambda hechos: hechos.update({'vehiculo': 'MiniVan'})},
    
    {'condiciones': lambda hechos: hechos['tipoDeVehiculo'] == 'automobil' and hechos['tamanio'] == 'grande' and hechos['num_puertas'] == 4,
    'accion': lambda hechos: hechos.update({'vehiculo': 'Vehiculo deportivo utilitario'})}
]

# Reglas para razonamiento hacia atrás
reglas_atras = {
    'Bicicleta': [
        {'condiciones': lambda hechos: hechos.get('tipoDeVehiculo') == 'cycle' and hechos.get('num_ruedas') == 2 and hechos.get('motor') == False}
    ],
    'Triciclo': [
        {'condiciones': lambda hechos: hechos.get('tipoDeVehiculo') == 'cycle' and hechos.get('num_ruedas') == 3 and hechos.get('motor') == False}
    ],
    'Motocicleta': [
        {'condiciones': lambda hechos: hechos.get('tipoDeVehiculo') == 'cycle' and hechos.get('num_ruedas') == 2 and hechos.get('motor') == True}
    ],
    'Formula 1': [
        {'condiciones': lambda hechos: hechos.get('tipoDeVehiculo') == 'automobil' and hechos.get('tamanio') == 'grande' and hechos.get('num_puertas') == 1}
    ],
    'Carro deportivo': [
        {'condiciones': lambda hechos: hechos.get('tipoDeVehiculo') == 'automobil' and hechos.get('tamanio') == 'pequenio' and hechos.get('num_puertas') == 2}
    ],
    'Sedan': [
        {'condiciones': lambda hechos: hechos.get('tipoDeVehiculo') == 'automobil' and hechos.get('tamanio') == 'mediano' and hechos.get('num_puertas') == 4}
    ],
    'MiniVan': [
        {'condiciones': lambda hechos: hechos.get('tipoDeVehiculo') == 'automobil' and hechos.get('tamanio') == 'mediano' and hechos.get('num_puertas') == 3}
    ],
    'Vehiculo deportivo utilitario': [
        {'condiciones': lambda hechos: hechos.get('tipoDeVehiculo') == 'automobil' and hechos.get('tamanio') == 'grande' and hechos.get('num_puertas') == 4}
    ],
    'cycle': [
        {'condiciones': lambda hechos: hechos.get('num_ruedas') < 4}
    ],
    'automobil': [
        {'condiciones': lambda hechos: hechos.get('num_ruedas') == 4 and hechos.get('motor') == True}
    ]
}
# Almacenar el resultado del razonamiento hacia adelante
vehiculo_inferido = None # Almacenar el vehículo inferido

# Función de razonamiento hacia adelante
def razonamiento_hacia_adelante(hechos, reglas):
    cambios = True
    proceso_texto = ""
    while cambios:
        cambios = False
        for regla in reglas:
            if regla['condiciones'](hechos):
                hechos_antes = hechos.copy()
                regla['accion'](hechos)
                if hechos_antes != hechos:
                    cambios = True
                    proceso_texto += f"Regla aplicada: {regla}\n"
    return hechos, proceso_texto

# Función de razonamiento hacia atrás
def razonamiento_hacia_atras(hechos, objetivo):
    if hechos.get('vehiculo') == objetivo:
        return True
    if objetivo in reglas_atras:
        for regla in reglas_atras[objetivo]:
            if regla['condiciones'](hechos):
                subobjetivos = [hecho for hecho in regla['condiciones']._code_.co_names if hecho not in hechos]
                for subobjetivo in subobjetivos:
                    if not razonamiento_hacia_atras(hechos, subobjetivo):
                        return False
                hechos['vehiculo'] = objetivo
                return True
    return False
hechos = {
    'num_ruedas': 4,
    'motor': True,
    'tamanio': 'grande',
    'num_puertas': 1,
    'tipoDeVehiculo': None,
    'vehiculo': None
}

# Función para mostrar preguntas una por una en ventanas flotantes
def hacer_pregunta(pregunta, opciones, siguiente_pregunta, conversion_func=None):
    def seleccionar_respuesta():
        respuesta = var.get()
        if conversion_func:
            hechos[siguiente_pregunta] = conversion_func(respuesta)  # Convertir la respuesta si es necesario
        else:
            hechos[siguiente_pregunta] = respuesta
        top.destroy()
        procesar_preguntas()

    top = tk.Toplevel(root)
    top.title("Pregunta")
    
    tk.Label(top, text=pregunta).pack()
    
    var = tk.StringVar(value=opciones[0])
    
    for opcion in opciones:
        tk.Radiobutton(top, text=opcion, variable=var, value=opcion).pack()

    tk.Button(top, text="Aceptar", command=seleccionar_respuesta).pack()

# Función para procesar cada pregunta en orden
def procesar_preguntas():
    if 'num_ruedas' not in hechos:
        hacer_pregunta("¿Cuántas ruedas tiene el vehículo?", ["2", "3", "4"], 'num_ruedas', conversion_func=int)  # Convertir a int
    elif 'motor' not in hechos:
        hacer_pregunta("¿Tiene motor?", ["Sí", "No"], 'motor', conversion_func=lambda x: x == "Sí")  # Convertir a bool
    elif hechos['num_ruedas'] == 4 and 'tamanio' not in hechos:
        hacer_pregunta("¿Qué tamaño tiene el vehículo?", ["pequeño", "mediano", "grande"], 'tamanio')
    elif hechos['num_ruedas'] == 4 and 'num_puertas' not in hechos:
        hacer_pregunta("¿Cuántas puertas tiene el vehículo?", ["1", "2", "3", "4"], 'num_puertas', conversion_func=int)  # Convertir a int
    else:
        ejecutar_inferencia()

# Ejecutar razonamiento basado en la selección del usuario
def ejecutar_inferencia():
    global vehiculo_inferido
    base_conocimiento = var_base.get()
    tipo_razonamiento = var_tipo.get()

    # Definir reglas según la base de conocimiento seleccionada
    reglas_seleccionadas = reglas_adelante if base_conocimiento == "Vehículos" else None

    proceso = ""
    if tipo_razonamiento == "Hacia adelante":
        # Ejecutar razonamiento hacia adelante
        proceso = razonamiento_hacia_adelante(hechos, reglas_seleccionadas)
        vehiculo_inferido = hechos.get('vehiculo', None)  # Almacenar el vehículo inferido
        resultado_texto.set(f"Resultado: {vehiculo_inferido if vehiculo_inferido else 'No se pudo inferir'}")
    else:  # Razonamiento hacia atrás
        if vehiculo_inferido is None:
            # Si no hay resultado anterior, primero ejecutar hacia adelante
            proceso = razonamiento_hacia_adelante(hechos, reglas_seleccionadas)
            vehiculo_inferido = hechos.get('vehiculo', None)  # Almacenar el vehículo inferido
        
        if vehiculo_inferido:
            # Tomar el resultado del razonamiento hacia adelante como objetivo
            objetivo = vehiculo_inferido
            if razonamiento_hacia_atras(hechos, objetivo):
                proceso += f"\nRazonamiento hacia atrás confirmado para: {objetivo}"
            else:
                proceso += f"\nNo se pudo confirmar el resultado hacia atrás."
        else:
            proceso += "\nNo hay un vehículo inferido para verificar hacia atrás."

    # Mostrar el proceso y resultado
    proceso_texto.set(f"Proceso:\n{proceso}")
    resultado_texto.set(f"Resultado: {hechos.get('vehiculo', 'No se pudo inferir')}")



# Función llamada al hacer clic en "Ejecutar"
def ejecutar():
    global  vehiculo_inferido
    
    # Verificar si es necesario reiniciar los hechos (nueva ejecución)
    if var_tipo.get() == "Hacia adelante":
        # Si el tipo de razonamiento es hacia adelante, limpiar los hechos para nueva inferencia
        hechos.clear()  # Reiniciamos los hechos
        resultado_hacia_adelante = None  # Reiniciamos el resultado hacia adelante
        vehiculo_inferido = None  # Resetear el vehículo inferido
        procesar_preguntas()  # Comenzar de nuevo la serie de preguntas
    
    elif var_tipo.get() == "Hacia atrás":
        # Si ya se hizo un razonamiento hacia adelante, tomamos su resultado
        if vehiculo_inferido:
            ejecutar_inferencia()  # Ejecutar inferencia sin limpiar los hechos
        else:
            # Si no se hizo razonamiento hacia adelante, lo hacemos primero
            ejecutar_inferencia()  # Primero hacemos razonamiento hacia adelante

# Crear la ventana principal
root = tk.Tk()
root.title("Sistema Experto de Vehículos")

# Variables de control para los widgets
var_base = tk.StringVar(value="Vehículos")
var_tipo = tk.StringVar(value="Hacia adelante")
resultado_texto = tk.StringVar(value="Resultado: ")
proceso_texto = tk.StringVar(value="Proceso:")

# Interfaz gráfica
tk.Label(root, text="Seleccione la base de conocimiento:").pack()
tk.Radiobutton(root, text="Vehículos", variable=var_base, value="Vehículos").pack()

tk.Label(root, text="Seleccione el tipo de razonamiento:").pack()
tk.Radiobutton(root, text="Hacia adelante", variable=var_tipo, value="Hacia adelante").pack()
tk.Radiobutton(root, text="Hacia atrás", variable=var_tipo, value="Hacia atrás").pack()

tk.Button(root, text="Ejecutar", command=ejecutar).pack()

tk.Label(root, textvariable=resultado_texto).pack()
tk.Label(root, textvariable=proceso_texto).pack()

# Ejecutar la interfaz
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\monts\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\monts\AppData\Local\Temp\ipykernel_1628\2387303704.py", line 125, in seleccionar_respuesta
    procesar_preguntas()
  File "C:\Users\monts\AppData\Local\Temp\ipykernel_1628\2387303704.py", line 150, in procesar_preguntas
    ejecutar_inferencia()
  File "C:\Users\monts\AppData\Local\Temp\ipykernel_1628\2387303704.py", line 164, in ejecutar_inferencia
    proceso = razonamiento_hacia_adelante(hechos, reglas_seleccionadas)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\monts\AppData\Local\Temp\ipykernel_1628\2387303704.py", line 85, in razonamiento_hacia_adelante
    if regla['condiciones'](hechos):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\monts\AppData\Local\Temp\ipykernel_1628\2387303704